Gerçek hayatta veriler her zaman düzgün gelmez. Verilerin eksikleri bulunabilir. Bu bölümde eksikleri bulunan veriyi işlemeyi görücez. Eksik veriler (missing data) için de null, NaN gibi ifadeleri kullanacağız.

## Eksikli Veriyi Düzeltme Yöntemleri

Genelde eksik veriler için iki seçeneğimiz vardır: Eksik verileri maskeleyerek yok gibi davranma veya eksik kısımları başka bir veriyle doldurma.

Eksik verileri maskelemek için eksik verileri belirten bir bool dizisi kullanabiliriz. Başka bir veri ile doldurmak istersek de;

+ yapacağımız işlemde sıkıntı çıkarmayacak, o veriye özgü değerler,
+ çok az bulunan ve/veya ayırt edici bir değer (örn. şehir yüz ölçümü verisi için -1),
+ kayıp verileri ifade etmek için kullanılan float türünde bir değer olan NaN (Not a Number) kullanabiliriz.

Gelgelelim; maskeleme yaptığımızda verilerin bir kısmını görmezden gelmiş oluruz, başka bir veri ile doldurduğumuzda da gereksiz CPU kullanımı gibi sorunlar çıkabilir. Eksik veriyi işlemede en doğru çözüm yoktur.

## Pandas'ta Kayıp Veri

Pandas, R dilindeki gibi her veri türü için ayrı kayıp veriyi belirten değişken bulundurabilirdi. Ancak bu hantallaşmasına sebep olurdu. Çünkü R dilinde 4 tip veri varken Pandas'ta çok daha fazladır. Mesela R dilinde bir tane integer türü varken Numpy'da, dolaylı olarak Pandas'ta, 14 tane vardır.

Numpy'da kayıp veri için maskeleme işlemi yapabilirsiniz. Pandas'ta da maskeleme yapılabilir ancak maskeleme işlemini kullanmak gerek hafıza kullanımında gerek veriyi işlemede kullanışlı olmadığından tercih edilmez, kayıp veri yerine başka veri koyma yoluna gidilir. Pandas'ta kayıp verileri float tipi olan NaN veya bir Python nesnesi olan None ile doldurabilirsiniz.

### NaN ve None Veri Tipi

Numpy dizisinde None veri türü tanımlarsanız dizinizin türü object olur. Bazen kullanmak gerekse de object dizi ile yapılan işlemler daha yavaş olur. Ayrıca verilerde None kullanırsanız; max, min, sum gibi fonksiyonlarda da hata alırsınız.

NaN ise float tipi bir değişken olduğundan dizinin türü float kalır ve float dizi ile yapılan işlemler object ile yapılanlara kıyasla daha hızlıdır. Ama burada da şöyle bir sorun var ki NaN ile yapılan tüm aritmetik işlemlerin sonucu yine NaN döner.

In [1]:
import numpy as np
import pandas as pd

5 + np.nan

nan

In [2]:
veri1 = np.array([1, 2, 3, np.nan])
veri1.sum()

nan

Bunun için Numpy'da NaN verileri görmezden gelmenizi sağlayan fonksiyonlar bulunur.

In [3]:
print(np.nansum(veri1), np.nanmax(veri1), np.nanmin(veri1))

6.0 3.0 1.0


### Pandas'ta NaN ve None

Pandas'ta NaN ile None çok benzer veriler olduğundan, dizi türü int veya float ise object olan None, float olan NaN'a çevrilir.

In [4]:
pd.Series([1, 2, 3, np.nan, None])

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
dtype: float64

Dizi türü int veya float değilse None none olarak kalır.

In [5]:
serim = pd.Series([1, np.nan, 'kelime', None])
serim

0         1
1       NaN
2    kelime
3      None
dtype: object

## Null Değerlerinin Yönetimi

Eksik verilerin üstesinden gelmek için kullanışlı birsürü metod vardır. Örnekleri `serim` dizisi üzerinden gösterelim.

In [6]:
serim

0         1
1       NaN
2    kelime
3      None
dtype: object

#### isnull()

Eksik verileri belirten bool dizi oluşturur.

In [7]:
serim.isnull()

0    False
1     True
2    False
3     True
dtype: bool

`notnull` fonksiyonu da bu bool dizisinin tersini verir.

#### dropna()

Verinin kayıp verilerinden arındırılmış kopyasını döndürür.

In [8]:
serim.dropna()

0         1
2    kelime
dtype: object

Dataframe'ler içinse `dropna` çeşitli parametrelerle kullanılabilir.

In [9]:
dtfr = pd.DataFrame([[1, np.nan, 2],
                     [2,3,5],
                     [np.nan, 4, 6]])
dtfr

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Öntanımlı olarak `dropna` fonksiyonu içinde NaN verilerin bulunduğu satırların tamamını siler.

In [10]:
dtfr.dropna()

,0,1,2
1,2.0,3.0,5


`axis` parametresine `columns` veya `1` girerek NaN değerlerin bulunduğu sütunları silebilirsiniz.

In [11]:
dtfr.dropna(axis='columns')

,2
0,2
1,5
2,6


Bu şekilde kullanışlı olabilecek verileri de kaybedebiliriz. Daha spesifik `dropna` yapmak için `how` parametresinde; `all` yazarak sadece tüm satırın NaN olduğu verileri veya `any` yazarak (öntanımlı değer) içinde en az bir NaN bulunduran satırları silebilirsiniz.

Diğer bir veri temizleme ölçütü olarak da `tresh` parametresine sayı vererek o satırın silinmemesi için bulundurması gereken minimum NaN olmayan değer miktarını belirtebilirsiniz.

#### fillna()

Eksik verileri başka verilerle doldurmak için `fillna` fonksiyonu kullanılır. Eksik verileri parametresine yazdığınız değerle değiştirir.

In [12]:
serim.fillna(99)

0         1
1        99
2    kelime
3        99
dtype: object

Doldurulacak veri için bir değer yerine `metod` da belirleyebilirsiniz. `"ffill"` her NaN değeri kendisinden bir önceki değer ile, `"bfill"` ise bir sonraki değer ile doldurur.

In [13]:
serim.fillna(method="ffill")

0         1
1         1
2    kelime
3    kelime
dtype: object

Dataframeler'de de aynı şekilde kullanılmakla beraber `axis` atayabilirsiniz. Tabiki NaN verileri bir önceki veri ile doldur dediğiniz dizide NaN, ilk değer değilse.

In [14]:
dtfr

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [15]:
dtfr.fillna(method="bfill", axis=1) # her NaN veriyi yatayda bir sonraki veri ile doldurur.

,0,1,2
0,1.0,2.0,2.0
1,2.0,3.0,5.0
2,4.0,4.0,6.0
